**INSTALLING REQ PACKAGES**

In [ ]:
!pip install weaviate
!pip install langchain
!pip install openai
!pip install weaviate-client
!pip install -U langchain-weaviate
!pip install langchain_community
!pip install unstructured
!pip install "unstructured[pdf]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.0
    Uninstalling langchain-core-1.0.0:
      Successfully uninstalled langchain-core-1.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.4 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-text-splitters 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
  Using cached langchain_core-1.0.0-py3-none-any.whl.metadata (3.4 kB)
Using c

In [ ]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import os
from weaviate.classes.init import Auth
import weaviate.classes.config as wc
from langchain_weaviate import WeaviateVectorStore
import weaviate
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

**SETTING KEYS**

In [ ]:
OPENAI_API_KEY =""
WEAVIATE_API_KEY = ""
WEAVIATE_URL = ""

**SETTING UP DATA**

In [ ]:
loader = DirectoryLoader("/content/pdf")
data = loader.load()

**TEXT SPLITTING**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs=text_splitter.split_documents(data)

In [ ]:
len(docs)

50

**EMBEDDING MODEL**

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

In [ ]:
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7f35e5ed2360>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7f35deb8a990>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

**INITIALIZING WEAVIEATE**

In [ ]:
# code from weaviete platform
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)
print(client.is_ready())

True


In [ ]:
# defining the schema from the platform snippet
client.collections.delete("AI_BOT")
client.collections.create(
    name="AI_BOT",
    description="document for the bot",
    vectorizer_config=wc.Configure.Vectorizer.text2vec_openai(
        model="ada"
    ),
    properties=[
        wc.Property(
            name="content",
            data_type=wc.DataType.TEXT
        )
    ]
)
vectorstore = WeaviateVectorStore(client=client, index_name="AI_BOT", text_key="content", embedding=embeddings)

**INITIALIZING THE VECTORSTORE**

In [ ]:
# generating the vectore and storing it in weaviate
text_meta_pair = [(doc.page_content,doc.metadata) for doc in docs]
text, meta = list(zip(*text_meta_pair))
vectorstore.add_texts(text, meta)

['a448c39d-0c04-42b3-8730-76bb30b9a4c6',
 '5179202a-8687-4e1a-be76-3297a432f7d0',
 '5281895f-d4e5-4b92-9046-ee0bca3618d6',
 '77bf962d-9536-4150-aa1f-fd29dbcfae3a',
 '91fb8ebc-e72d-4e12-80e3-06a03f226b65',
 'fd43106e-ed26-46b7-b24e-ae1543273638',
 '5e5b67f8-6ff1-4c2f-b2e2-b6a2fc72c14d',
 '3c807a5d-e3db-4b60-8acb-6ac7874bae77',
 'fd245421-c9af-4d25-b814-8d219f855523',
 '9657e2fc-d3da-4c21-9c1c-b061515c6e15',
 '90f41c14-28ae-49ad-ad98-e0fc65a55f3b',
 'b68ee11c-c786-4b31-8a76-e3fa519ee15d',
 '6b1a90c7-dcb6-4ec1-b13b-f8641e68eb4e',
 'e546fd49-8bef-4927-b0eb-1a6b8ba85028',
 'c7d70458-ea9d-442e-8f5f-d2f11fb6caa0',
 'ce84f0cf-eb2b-4424-9ee5-2f26217fd3c0',
 'bc5a354c-1fef-4c0c-8e15-0f3105267f57',
 '18c52e97-1b75-47cb-95d3-86f8642d82f6',
 '7eef7c4b-0b00-4954-8598-b761030050bb',
 '6d5ee4f3-971d-444f-8786-8ce80989ac4a',
 '299721a8-6acb-4167-b7b9-1479360f7966',
 '50d96102-5db0-4c12-9c71-0dd8cfabe9ee',
 '310727b9-73bb-4ba6-9619-ad2af6f06de8',
 'a3658fe1-3a21-4dce-b0c1-1edc0b65e1d1',
 '618f32a4-96a2-

**SIMILARITY SEARCH IE QUERY**

In [ ]:
q1 = " what is yolo"
docs = vectorstore.similarity_search(q1, k=3)
docs

[Document(metadata={'source': '/content/pdf/Redmon_You_Only_Look_CVPR_2016_paper.pdf'}, page_content='Table 3: PASCAL VOC 2012 Leaderboard. YOLO compared with the full comp4 (outside data allowed) public leaderboard as of November 6th, 2015. Mean average precision and per-class average precision are shown for a variety of detection methods. YOLO is the only real-time detector. Fast R-CNN + YOLO is the forth highest scoring method, with a 2.3% boost over Fast R-CNN.\n\nmAP increases by 3.2% to 75.0%. We also tried combining the top Fast R-CNN model with several other versions of Fast R-CNN. Those ensembles produced small increases in mAP between .3 and .6%, see Table 2 for details.\n\nthe test data can diverge from what the system has seen be- fore [3]. We compare YOLO to other detection systems on the Picasso Dataset [12] and the People-Art Dataset [3], two datasets for testing person detection on artwork.'),
 Document(metadata={'source': '/content/pdf/Redmon_You_Only_Look_CVPR_2016_pa

**INITIALIZING LLM**

In [ ]:
# as chains is depriceated so using FAISS
!pip install langchain langchain-openai langchain-community pypdf faiss-cpu tiktoken

In [ ]:
vectorstore = FAISS.from_documents(docs, embeddings)
query = "what is VOC 2007 Error Analysis"
results = vectorstore.similarity_search(query, k=3)
llm = ChatOpenAI(temperature=0)
context = "\n\n".join([doc.page_content for doc in results])
prompt = f"Answer the question based on the context below.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"
response = llm.invoke(prompt)
print("LLM Answer:", response.content)


LLM Answer: VOC 2007 Error Analysis is a detailed breakdown of results on the PASCAL VOC 2007 dataset, comparing YOLO to Fast R-CNN and examining the differences between them in terms of errors.
